In [2]:
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import dlib
import cv2
import glob
import progressbar
import numpy as np
from sklearn import svm
import pickle
from sklearn.metrics import accuracy_score

c:\Users\thang\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

In [4]:
def get_areas(img):
    #Find eyes in faces
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    imutils.resize(image=img, width=450)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        return ear

In [ ]:
dataset_path = "./Dataset/CEW/"
imagesclosed = [cv2.imread(file) for file in
                glob.glob(dataset_path + "ClosedFace/" + "*.jpg")]
imagesopen = [cv2.imread(file) for file in glob.glob(dataset_path + "/OpenFace/" + "*.jpg")]

X = []
Y = []
X_test = []
Y_test = []

bar = progressbar.ProgressBar(maxval=len(imagesclosed),
                              widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for idx, im in enumerate(imagesclosed):
    bar.update(idx + 1)
    e = get_areas(im)
    if e is not None: #Avoid dirty data derived by faceland detector
        if idx > len(imagesclosed) * 0.9: #Take 10% of images with closed eyes as test
            X_test.append(e)
            Y_test.append(0)
        else: #Use the rest for train
            X.append(e)
            Y.append(0)
bar.finish()

bar = progressbar.ProgressBar(maxval=len(imagesopen),
                              widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for idx, im in enumerate(imagesopen):
    bar.update(idx + 1)
    e = get_areas(im)
    if e is not None:  # Avoid dirty data derived by faceland detector
        if idx > len(imagesopen) * 0.9:  # Take 10% of images with closed eyes as test
            X_test.append(e)
            Y_test.append(1)
        else:  # Use the rest for train
            X.append(e)
            Y.append(1)
bar.finish()

X = np.asarray(X).reshape(-1, 1)
Y = np.asarray(Y)
X_test = np.asarray(X_test).reshape(-1, 1)
Y_test = np.asarray(Y_test)

model = svm.SVC(C=1000, gamma=0.1)
model.fit(X, Y)

In [10]:
#Evaluate
predicted_labels = model.predict(X_test)

# get the accuracy
print(accuracy_score(Y_test, predicted_labels))
filename = 'svm_C=1000_gamma=0.1.sav'
pickle.dump(model, open(filename, 'wb'))


0.8868778280542986


In [5]:
image_open = [cv2.imread(file) for file in glob.glob("./image_open.jpg")]
for idx, im in enumerate(image_open):
    e_open = get_areas(im)

In [6]:
image_closed = [cv2.imread(file) for file in glob.glob("./image_closed.jpg")]
for idx, im in enumerate(image_closed):
    e_closed = get_areas(im)

In [7]:
# Load the model from disk
loaded_model = pickle.load(open('./svm_C=1000_gamma=0.1.sav', 'rb'))
predict1 = loaded_model.predict([[e_open]])
predict1 # 1 is open # 0 is closed

array([1])

In [8]:
predict2 = loaded_model.predict([[e_closed]])
print(predict2)

[0]
